# Milestone 2

---

Milestones ini dibuat guna mengevaluasi pembelajaran pada Hacktiv8 Data Science Full Time Program Phase 2 khususnya pada Deep Learning.

## Introduction

By [Rifky Aliffa](https://github.com/Penzragon)

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/The_Rim_Fire_in_the_Stanislaus_National_Forest_near_in_California_began_on_Aug._17%2C_2013-0004.jpg/1200px-The_Rim_Fire_in_the_Stanislaus_National_Forest_near_in_California_began_on_Aug._17%2C_2013-0004.jpg)

### Dataset

Pada project ini dataset yang digunakan adalah berbagai macam gambar kebakaran dan api. Dataset dapat dilihat di [Kaggle](https://www.kaggle.com/datasets/phylake1337/fire-dataset).

### Objectives

**Milestones 2** ini dibuat guna mengevaluasi Pembelajaran Phase 2 :

- Mampu memahami konsep Computer Vision/NLP.
- Mampu mempersiapkan data untuk digunakan dalam model Computer Vision/NLP.
- Mampu mengimplementasikan Artificial Neural Network dengan data yang ada.
- Mampu menganalisis dan menjelaskan layer yang dibuat.

Dan tujuan utama dari project ini adalah untuk membuat model CNN yang mampu mendeteksi apakah terjadi kebakaran atau tidak.

## Import Libraries

Pada project ini library yang akan digunakan adalah **Pandas**, **Numpy**, **Matplotlib**, **Seaborn**, **Tensorflow**, **Keras**, dan **Scikit-Learn**.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Data Loading

Membuat dataframe yang berisi path dari gambar berikut labelnya.

In [3]:
# Create an empty dataframe
fire = pd.DataFrame(columns=['image', 'label'])

# Loop through the folders in the data directory
for dirpath, dirnames, filenames in os.walk('dataset/fire_dataset/'):
    # Loop through the fire images
    for filename in filenames:
        fire = fire.append({'image': os.path.join(dirpath, filename), 'label': 'fire'}, ignore_index=True)

    # Loop through the non-fire images
    for filename in filenames:
        fire = fire.append({'image': os.path.join(dirpath, filename), 'label': 'non_fire'}, ignore_index=True)

# Shuffle the dataframe
fire = fire.sample(frac=1).reset_index(drop=True)

In [4]:
fire.head(20)

,image,label
0,dataset/fire_dataset/fire_images\fire.68.png,non_fire
1,dataset/fire_dataset/fire_images\fire.373.png,non_fire
2,dataset/fire_dataset/fire_images\fire.721.png,non_fire
3,dataset/fire_dataset/non_fire_images\non_fire....,non_fire
4,dataset/fire_dataset/fire_images\fire.620.png,non_fire
5,dataset/fire_dataset/fire_images\fire.351.png,fire
6,dataset/fire_dataset/fire_images\fire.735.png,fire
7,dataset/fire_dataset/fire_images\fire.315.png,non_fire
8,dataset/fire_dataset/non_fire_images\non_fire....,fire
9,dataset/fire_dataset/fire_images\fire.145.png,non_fire


## Exploratory Data Analysis (EDA)